In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
def size_on_day(d, day):
    d_day_x = d.loc[d.Day == day]
    d_day_x = d_day_x[['MID', 'Size']]
    colname = 'Day-' + str(day) + '-Size'
    d_day_x = d_day_x.rename(columns = {'Size': colname})
    return d_day_x

In [2]:
DATA = "~/Documents/research/tansey/datasets/supplemental_table_3_-_raw_volume_data_with_control.csv"

In [3]:
df = pd.read_csv(DATA)
df.head()

,Study,Study_ID,Group,Drug,Control,MID,Day,Size
0,J000100672,42396T_2CAOS,3,Cisplatin,0.0,396-11,0.0,212.49
1,J000100672,42396T_2CAOS,3,Cisplatin,0.0,396-11,3.0,322.60
2,J000100672,42396T_2CAOS,3,Cisplatin,0.0,396-11,6.0,368.00
3,J000100672,42396T_2CAOS,3,Cisplatin,0.0,396-11,10.0,538.42
4,J000100672,42396T_2CAOS,3,Cisplatin,0.0,396-11,13.0,743.13


In [6]:
l1 = len(df)
l1

15433

In [7]:
df = df[df['Study'].notna()]

In [9]:
l2 = len(df)
for col in df.columns:
    assert(len(df.loc[df[col].isna()]) == 0)
print('dropped ' + str(l1 - l2) + ' rows with nans')

dropped 638 rows with nans


In [10]:
df = df[df['Day'] >= 0]

In [11]:
l3 = len(df)
print('dropped ' + str(l2 - l3) + ' rows with Day < 0')

dropped 798 rows with Day < 0


In [12]:
df = df[['Study_ID', 'Drug', 'Control', 'MID', 'Day', 'Size']]
df.head()

,Study_ID,Drug,Control,MID,Day,Size
0,42396T_2CAOS,Cisplatin,0.0,396-11,0.0,212.49
1,42396T_2CAOS,Cisplatin,0.0,396-11,3.0,322.60
2,42396T_2CAOS,Cisplatin,0.0,396-11,6.0,368.00
3,42396T_2CAOS,Cisplatin,0.0,396-11,10.0,538.42
4,42396T_2CAOS,Cisplatin,0.0,396-11,13.0,743.13


In [13]:
old_len = len(df)

In [15]:
mids = pd.DataFrame(df.MID).drop_duplicates()
len(mids)

1775

In [16]:
mids.head()

,MID
0,396-11
9,396-20
18,396-21
27,396-31
36,396-32


In [17]:
start_day = 0
end_day = 28
start_size = size_on_day(df, start_day)
print(len(start_size))
end_size = size_on_day(df, end_day)
print(len(end_size))
print(len(mids))
mids = mids.merge(start_size, on='MID', validate='one_to_one')
print(len(mids))
mids = mids.merge(end_size, on='MID', validate='one_to_one')
print(len(mids))

834
656
1775
834
399


In [18]:
print(mids)

        MID  Day-0-Size  Day-28-Size
0    173-17       90.62       459.42
1    173-18      221.97       483.71
2    173-20      100.00       346.87
3    173-25       91.73       418.22
4    173-26       82.65       402.09
..      ...         ...          ...
394  179_39      118.98       117.85
395  179_42      294.91       316.28
396  179_44      114.02       138.32
397  179_52       94.59        69.76
398  179_54      202.29       229.38

[399 rows x 3 columns]


In [19]:
print(len(mids))
print(len(df))

399
13997


In [20]:
df = df[['Study_ID', 'Drug', 'Control', 'MID']].merge(mids, on='MID', validate='many_to_one')

In [21]:
len(df)

3557

In [22]:
df = df.drop_duplicates()

In [23]:
len(df)

411

In [24]:
df.head()

,Study_ID,Drug,Control,MID,Day-0-Size,Day-28-Size
0,JW215_07T101173,Cisplatin,0.0,173-17,90.62,459.42
9,JW215_07T101173,Cisplatin,0.0,173-18,221.97,483.71
18,JW215_07T101173,Cisplatin,0.0,173-20,100.00,346.87
27,JW215_07T101173,Cisplatin,0.0,173-25,91.73,418.22
36,JW215_07T101173,Cisplatin,0.0,173-26,82.65,402.09


In [25]:
treat_mids = df.loc[df['Control'] == 1.0].MID.unique()
control_mids = df.loc[df['Control'] == 0.0].MID.unique()
intersection_mids = list(set(treat_mids).intersection(set(control_mids)))

In [26]:
df = df.loc[df['MID'].isin(intersection_mids) == False]

In [27]:
len(df)

400

In [28]:
treat = df.loc[df['Control'] == 0.0]
control = df.loc[df['Control'] == 1.0]

In [29]:
control.head()

,Study_ID,Drug,Control,MID,Day-0-Size,Day-28-Size
1296,JW215_07T101173,D5W,1.0,173-02,172.49,882.80
1305,JW215_07T101173,D5W,1.0,173-03,85.55,1660.15
1314,JW215_07T101173,D5W,1.0,173-06,98.16,624.50
1323,JW215_07T101173,D5W,1.0,173-09,124.32,419.17
1332,JW215_07T101173,D5W,1.0,173-12,122.35,869.84


In [30]:
control['relative_vol'] = control['Day-28-Size'] / control['Day-0-Size']

<ipython-input-30-9a994a089c4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['relative_vol'] = control['Day-28-Size'] / control['Day-0-Size']


In [31]:
c_mean_vol = control.groupby('Study_ID')['Day-28-Size'].mean().to_frame(name='c_mean_vol').reset_index()
c_med_vol= control.groupby('Study_ID')['Day-28-Size'].median().to_frame(name='c_med_vol').reset_index()
c_mean_rel_vol = control.groupby('Study_ID')['relative_vol'].mean().to_frame(name='c_mean_rel_vol').reset_index()
c_med_rel_vol = control.groupby('Study_ID')['relative_vol'].median().to_frame(name='c_med_rel_vol').reset_index()

In [32]:
control_study_ID = control.Study_ID.to_frame().drop_duplicates()

In [33]:
control_study_ID = control_study_ID.merge(c_mean_vol, on='Study_ID', validate='one_to_one')
control_study_ID = control_study_ID.merge(c_med_vol, on='Study_ID', validate='one_to_one')
control_study_ID = control_study_ID.merge(c_mean_rel_vol, on='Study_ID', validate='one_to_one')
control_study_ID = control_study_ID.merge(c_med_rel_vol, on='Study_ID', validate='one_to_one')

In [34]:
control_study_ID

,Study_ID,c_mean_vol,c_med_vol,c_mean_rel_vol,c_med_rel_vol
0,JW215_07T101173,692.697778,624.500,6.050728,3.927095
1,JW215_11TBR0869,921.060000,921.060,9.907067,9.907067
2,JW226_03TJ102630,706.787500,680.605,6.070190,5.436284
3,JW215_12TBR0555,407.373750,355.890,3.215851,3.383921
4,JW216_05TJ105006,591.220000,591.220,5.402723,5.402723
5,JW217_04TLG1049,374.336667,370.390,3.518140,3.917126
6,JW217_05TLG1265,543.057000,513.720,5.671043,4.844673
7,JW217_06TLG627PE,688.201250,569.440,5.950491,5.767366
8,JW217T_LG0551,593.265556,502.020,5.687443,5.807873
9,JW217T_LG0978,1156.365000,1155.425,12.116899,11.342707


In [35]:
treat.head()

,Study_ID,Drug,Control,MID,Day-0-Size,Day-28-Size
0,JW215_07T101173,Cisplatin,0.0,173-17,90.62,459.42
9,JW215_07T101173,Cisplatin,0.0,173-18,221.97,483.71
18,JW215_07T101173,Cisplatin,0.0,173-20,100.00,346.87
27,JW215_07T101173,Cisplatin,0.0,173-25,91.73,418.22
36,JW215_07T101173,Cisplatin,0.0,173-26,82.65,402.09


In [36]:
treat['t_rel_vol'] = treat['Day-28-Size'] / treat['Day-0-Size']

<ipython-input-36-4c36159ed53f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treat['t_rel_vol'] = treat['Day-28-Size'] / treat['Day-0-Size']


In [37]:
treat.head()

,Study_ID,Drug,Control,MID,Day-0-Size,Day-28-Size,t_rel_vol
0,JW215_07T101173,Cisplatin,0.0,173-17,90.62,459.42,5.069742
9,JW215_07T101173,Cisplatin,0.0,173-18,221.97,483.71,2.179168
18,JW215_07T101173,Cisplatin,0.0,173-20,100.00,346.87,3.468700
27,JW215_07T101173,Cisplatin,0.0,173-25,91.73,418.22,4.559250
36,JW215_07T101173,Cisplatin,0.0,173-26,82.65,402.09,4.864973


In [38]:
m = treat.merge(control_study_ID, on='Study_ID', validate='many_to_one')
len(m)

261

In [39]:
m.head()

,Study_ID,Drug,Control,MID,Day-0-Size,Day-28-Size,t_rel_vol,c_mean_vol,c_med_vol,c_mean_rel_vol,c_med_rel_vol
0,JW215_07T101173,Cisplatin,0.0,173-17,90.62,459.42,5.069742,692.697778,624.5,6.050728,3.927095
1,JW215_07T101173,Cisplatin,0.0,173-18,221.97,483.71,2.179168,692.697778,624.5,6.050728,3.927095
2,JW215_07T101173,Cisplatin,0.0,173-20,100.00,346.87,3.468700,692.697778,624.5,6.050728,3.927095
3,JW215_07T101173,Cisplatin,0.0,173-25,91.73,418.22,4.559250,692.697778,624.5,6.050728,3.927095
4,JW215_07T101173,Cisplatin,0.0,173-26,82.65,402.09,4.864973,692.697778,624.5,6.050728,3.927095


In [40]:
m['trad_T_C'] = m['Day-28-Size'] / m['c_med_vol']
m['rel_T_C'] = m['t_rel_vol'] / m['c_med_rel_vol']

In [41]:
m.head()

,Study_ID,Drug,Control,MID,Day-0-Size,Day-28-Size,t_rel_vol,c_mean_vol,c_med_vol,c_mean_rel_vol,c_med_rel_vol,trad_T_C,rel_T_C
0,JW215_07T101173,Cisplatin,0.0,173-17,90.62,459.42,5.069742,692.697778,624.5,6.050728,3.927095,0.735661,1.290965
1,JW215_07T101173,Cisplatin,0.0,173-18,221.97,483.71,2.179168,692.697778,624.5,6.050728,3.927095,0.774556,0.554906
2,JW215_07T101173,Cisplatin,0.0,173-20,100.00,346.87,3.468700,692.697778,624.5,6.050728,3.927095,0.555436,0.883274
3,JW215_07T101173,Cisplatin,0.0,173-25,91.73,418.22,4.559250,692.697778,624.5,6.050728,3.927095,0.669688,1.160973
4,JW215_07T101173,Cisplatin,0.0,173-26,82.65,402.09,4.864973,692.697778,624.5,6.050728,3.927095,0.643859,1.238822


In [42]:
m.trad_T_C.describe()

count    261.000000
mean       0.512597
std        0.372205
min        0.042611
25%        0.216460
50%        0.419933
75%        0.735661
max        2.164236
Name: trad_T_C, dtype: float64

In [43]:
m.rel_T_C.describe()

count    261.000000
mean       0.510889
std        0.352287
min        0.040970
25%        0.252664
50%        0.463138
75%        0.715721
max        2.258838
Name: rel_T_C, dtype: float64

In [44]:
m.head()

,Study_ID,Drug,Control,MID,Day-0-Size,Day-28-Size,t_rel_vol,c_mean_vol,c_med_vol,c_mean_rel_vol,c_med_rel_vol,trad_T_C,rel_T_C
0,JW215_07T101173,Cisplatin,0.0,173-17,90.62,459.42,5.069742,692.697778,624.5,6.050728,3.927095,0.735661,1.290965
1,JW215_07T101173,Cisplatin,0.0,173-18,221.97,483.71,2.179168,692.697778,624.5,6.050728,3.927095,0.774556,0.554906
2,JW215_07T101173,Cisplatin,0.0,173-20,100.00,346.87,3.468700,692.697778,624.5,6.050728,3.927095,0.555436,0.883274
3,JW215_07T101173,Cisplatin,0.0,173-25,91.73,418.22,4.559250,692.697778,624.5,6.050728,3.927095,0.669688,1.160973
4,JW215_07T101173,Cisplatin,0.0,173-26,82.65,402.09,4.864973,692.697778,624.5,6.050728,3.927095,0.643859,1.238822


In [45]:
sid_list = list(m.Study_ID.unique())
print(sid_list)

['JW215_07T101173', 'JW215_11TBR0869', 'JW226_03TJ102630', 'JW215_12TBR0555', 'JW216_05TJ105006', 'JW217_04TLG1049', 'JW217_05TLG1265', 'JW217_06TLG627PE', 'JW217T_LG0551', 'JW217T_LG0978', 'JW217T_LG1102', 'JW217T2_LG1191', 'JW223_03TPR1996F', 'JW224_01TOV1828', 'LG0645T_JW217', 'LG0796T_JW217', 'LG0830Ta_JW217', 'LG1179T_JW217', 'JW217_08TLG0801']


In [46]:
SAVE_DIR = '/Users/michelameister/Documents/research/tansey/results/2023-03-21/'

In [47]:
for sid in sid_list:
    sf = m.loc[m['Study_ID'] == sid]
    vals = list(sf['trad_T_C'])
    main_title = sid + ', N = ' + str(len(vals))
    plt.scatter(vals, [0]*len(vals))
    plt.title(main_title)
    plt.xlabel('treatment vol @28 / control vol @28')
    fn = SAVE_DIR + 'trad_T_C/' + sid + '.jpg'
    plt.savefig(fn)
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [48]:
for sid in sid_list:
    sf = m.loc[m['Study_ID'] == sid]
    vals = list(sf['t_rel_vol'])
    main_title = sid + ', N = ' + str(len(vals))
    plt.scatter(vals, [0]*len(vals))
    plt.title(main_title)
    plt.xlabel('treatment vol @28 / treatment vol @0')
    fn = SAVE_DIR + 't_rel_vol/' + sid + '.jpg'
    plt.savefig(fn)
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [49]:
for sid in sid_list:
    sf = m.loc[m['Study_ID'] == sid]
    vals = list(sf['rel_T_C'])
    main_title = sid + ', N = ' + str(len(vals))
    plt.scatter(vals, [0]*len(vals))
    plt.title(main_title)
    plt.xlabel('(treatment vol @28 / treatment vol @0) / (control vol @28 / control vol @0)')
    fn = SAVE_DIR + 'rel_T_C/' + sid + '.jpg'
    plt.savefig(fn)
    plt.clf()

<Figure size 432x288 with 0 Axes>